In [ ]:
import math, random
import numpy as np
import scipy.fftpack as fft

In [ ]:
# x = np.array([random.random() + 0j for _ in range(16)])
x = np.array([
	0.70203658, 0.30785784, 0.80697642, 0.2063156 ,
	0.74611309, 0.44949445, 0.58790534, 0.94034123,
	0.86815133, 0.78308922, 0.51704855, 0.58557402,
	0.49798021, 0.43429341, 0.52435585, 0.47455634,
])
N, n = len(x), len(x)//2

x, fft.dct(x)

In [ ]:
# Single real FFT
xe, xo = x[0::2], x[1::2]
x2 = xe + xo*1j
Z = fft.fft(x2)

Xe = np.zeros(n, complex)
Xo = np.zeros(n, complex)
for i in range(n):
	Xe[i] = (Z[i] + Z[-i].conj())*(0.5 - 0.0j)
	Xo[i] = (Z[i] - Z[-i].conj())*(0.0 - 0.5j)

X = np.zeros(N, complex)
w, wm = 1, math.e**(-2j*math.pi/N)
X[n] = Xe[0] - Xo[0]
for i in range(n):
	Xi = Xe[i] + Xo[i]*w
	X[i], X[-i] = Xi, Xi.conj()
	w *= wm


np.round(X - fft.fft(x), 4)
# Z, X, Xe, Xo

In [ ]:
# Single real iFFT (TODO not implemented)
X = fft.fft(x)

Xe = np.zeros(n, complex)
Xo = np.zeros(n, complex)
for i in range(n):
	Xe[i] = (X[i] + X[-i].conj())/2
	Xo[i] = (X[i] - X[-i].conj())/2*math.e**(2j*math.pi*i/N)

Z = Xe + Xo*1j
# Z - fft.fft(x)

In [ ]:
# DCT via FFT
xdct = np.zeros(2*N, complex)
xdct[ 0: N: 1] = x
xdct[-1:-N-1:-1] = x

Xdct = fft.fft(xdct)[:N]

# Apply half sample rotation.
w, wm = 1, math.e**(-0.5j*math.pi/N)
for i in range(N):
	Xdct[i] *= w
	w *= wm

np.round(Xdct - fft.dct(x), 10)

In [ ]:
# iDCT via iFFT
Xdct = fft.dct(x) + 0j

Xtmp = np.zeros(2*N, complex)

# Apply half sample rotation.
w, wm = 1, math.e**(0.5j*math.pi/N)
for i in range(N):
	X = Xdct[i]*w
	Xtmp[i] = X
	Xtmp[-i] = X.conj()
	w *= wm

np.round(x - fft.ifft(Xtmp)[:N], 10)

In [ ]:
# DCT via real FFT
xdct = np.zeros(2*N, complex)
xdct[ 0: N: 1] = x
xdct[-1:-N-1:-1] = x

xe, xo = xdct[0::2], xdct[1::2]
xdct = xe + xo*1j
Z = fft.fft(xdct)

Xe = np.zeros(N, complex)
Xo = np.zeros(N, complex)
Xdct = np.zeros(N, complex)
w, wm = 1, math.e**(-0.5j*math.pi/N)
for i in range(N):
	Xe[i] = (Z[i] + Z[-i].conj())*(0.5 - 0.0j)
	Xo[i] = (Z[i] - Z[-i].conj())*(0.0 - 0.5j)
	Xdct[i] = (Xe[i] + Xo[i]*w*w)*w
	w *= wm

np.round(Xdct - fft.dct(x), 10)

In [ ]:
# iDCT via real iFFT
Xdct = fft.dct(x) + 0j

# Apply half sample rotation.
w, wm = 1, math.e**(0.5j*math.pi/N)
for i in range(N):
	Xdct[i] *= w
	# print(w)
	w *= wm

Z = np.zeros(N, complex)
w, wm = 1, math.e**(1j*math.pi/N)
for i in range(N):
	X0, X1 = Xdct[i], Xdct[-i]
	Xe = (X0 + X1.conj())
	Xo = (X0 - X1.conj())*w
	Z[i] = Xe + Xo*1j
	w *= wm
Z[0] *= complex(0.5, 0.5)

z = fft.fft(Z.conj()).conj()*0.5/N
np.round(xdct - z, 10)

In [ ]:
x = np.array([random.random() + 0j for _ in range(1 << 16)])
for _ in range(4000):
	x = fft.fft(x)
	x = fft.ifft(x)